In [6]:
"""
Description: Model that conducts Line of Balance.
Assignment:  A5 - Line of Balance (Question 1 and 2)

Author: Mehulkumar JAriwala And Akshaykumar Dhorajiya

Due Date: 8TH DEC, 2021

Language: PYTHON

To Compile: In the command line, please run the following command:

        A5_Mehul_Jariwala_And_Akshay_Dhorajiya.ipynb

Not Addressed: N/A - all requirements of the assignments are addressed

Input:  1. Change the CSV filepath on when inserting new CSV files.
        2. Please ensure the new CSV files are in the SAME DIRECTORY.
      

Output: The following output is shown:
        1. Using matplotlib, png(s) are generated as well as saved in the directory: ./Q1_Plots/
            Plots include:
                - Q1 LOB Diagram.png - LOB Diagram of each activity in terms of productivity (units/days)
                - Project Duration and Distancec from given deadline is shown at top of the LOB Diagram
                - Q1 LOB Table.png - Tabular format of LOB Analysis

Algorithm: The following algorithm is used:
        1. Class Node to store Duration, Buffer, ES, EF, LS, LF, TF, LF, Th_teams, Ac_tems,
                                                     Th_prod, Ac_prod, FUS, FUL, LUS, LUF

        2. A recursive forward and backward implementation is used to ensure that the formulae to calculate the
           activity parameters is satisfied.

        3. Initial Project Calculations:
            3.1. Using the given delays, calculate the initial project duration
            3.2. To calculate the initial project productivity:
                3.2.1. initial_project_productivity = (desired_duration - initial_project_duration) / (units-1)
            3.3. Using the initial_project_productivity, calculate the project_duration by performing LOB Analysis.

        4. Use a Binary-Search-Like implementation for trial/error to get the project_duration (step 3.3) as close to
           desired_project_duration.


Known Bugs: N/A


  
"""
from __future__ import annotations
import json
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.patches as patches
import numpy as np
from numpy.core.arrayprint import StructuredVoidFormat, _guarded_repr_or_str
from numpy.core.fromnumeric import size
from numpy.lib.arraypad import pad
import os
from pandas.plotting import table
import pandas as pd
import random
import sys
from typing import SupportsComplex


class Node:

    # Initializer
    def __init__(self, activity, duration, buffer):
        self.activity = activity
        self.duration = duration
        self.buffer = buffer

        self.es = None
        self.ef = None
        self.ls = None
        self.lf = None
        self.tf = None
        self.ff = None

        self.prev = []
        self.next = []

        self.th_teams = None
        self.ac_teams = None
        self.th_prod = None
        self.ac_prod = None
        self.fus = None
        self.fuf = None
        self.lus = None
        self.luf = None

    # Function to print the Node object
    def __repr__(self) -> str:
        return "{" \
               f"\"Activity\":\"{self.activity}\", " \
               f"\"Duration\":\"{self.duration}\", " \
               f"\"Buffer\":\"{self.buffer}\", " \
               f"\"ES\":\"{self.es}\", " \
               f"\"EF\":\"{self.ef}\", " \
               f"\"LS\":\"{self.ls}\", " \
               f"\"LF\":\"{self.lf}\", " \
               f"\"TF\":\"{self.tf}\", " \
               f"\"FF\":\"{self.ff}\", " \
               f"\"prev\":\"{list(map(lambda node: node.activity, self.prev))}\", " \
               f"\"next\":\"{list(map(lambda node: node.activity, self.next))}\", " \
               f"\"Th_teams\":\"{self.th_teams}\", " \
               f"\"Ac_teams\":\"{self.ac_teams}\", " \
               f"\"Th_prod\":\"{self.th_prod}\", " \
               f"\"Ac_prod\":\"{self.ac_prod}\", " \
               f"\"FUS\":\"{self.fus}\", " \
               f"\"FUF\":\"{self.fuf}\", " \
               f"\"LUS\":\"{self.lus}\", " \
               f"\"LUF\":\"{self.luf}\" " \
               "}\n"

    def get_duration(self):
        return self.duration

    def set_duration(self, duration):
        self.duration = duration

    def get_buffer(self):
        return self.buffer

    def get_ES(self):
        return self.es

    def set_ES(self, ES):
        self.es = ES

    def get_EF(self):
        return self.ef

    def set_EF(self, EF):
        self.ef = EF

    def get_LS(self):
        return self.ls

    def get_LF(self):
        return self.lf

    def get_FF(self):
        return self.ff

    def get_successors(self):
        return self.next

    def get_activity(self):
        return self.activity

    def get_TF(self):
        return self.tf

    def set_th_teams(self, th_teams):
        self.th_teams = th_teams

    def get_th_teams(self):
        return self.th_teams

    def set_ac_teams(self, ac_teams):
        self.ac_teams = ac_teams

    def get_ac_teams(self):
        return self.ac_teams

    def set_th_prod(self, th_prod):
        self.th_prod = th_prod

    def get_th_prod(self):
        return self.th_prod

    def set_ac_prod(self, ac_prod):
        self.ac_prod = ac_prod

    def get_ac_prod(self):
        return self.ac_prod

    def get_prev(self):
        return self.prev

    def get_next(self):
        return self.next

    def set_fus(self, fus):
        self.fus = fus

    def get_fus(self):
        return self.fus

    def set_fuf(self, fuf):
        self.fuf = fuf

    def get_fuf(self):
        return self.fuf

    def set_lus(self, lus):
        self.lus = lus

    def get_lus(self):
        return self.lus

    def set_luf(self, luf):
        self.luf = luf

    def get_luf(self):
        return self.luf


# Read the csv file and returns a pandas df object
def read_csv(path):
    return pd.read_csv(path)


# Generates nodes from the input csv
def generate_nodes(csv, prod) -> list[str, Node]:
    # Add the Start and End nodes manually
    nodes = {
        "Start": Node(activity="Start", duration=0, buffer=0),
        "End": Node(activity="End", duration=0, buffer=0)
    }

    csv = csv.rename(columns=lambda x: x.strip())
    columns = csv.columns

    # Iterate through the dataframe
    for index, row in csv.iterrows():
        if row["Activity"] not in nodes:
            # Add new nodes
            activity = row["Activity"]
            duration = row["Duration"]
            buffer = str(row["Buffer"])

            if buffer == "nan":
                buffer = 0

            buffer = float(buffer)
            # Create a new node
            if prod:
                node = Node(activity, duration + buffer, buffer)
            else:
                node = Node(activity, duration, buffer)

            # Add node to nodes
            nodes[activity] = node

    # Iterate to keep track of prev and next nodes
    for index, row in csv.iterrows():
        # Dependency list
        current = row["Activity"].strip()

        if row["Predecessors"] is np.nan:
            continue

        dependents = row["Predecessors"].strip().split(",")
        for dependent in dependents:
            dependent = dependent.strip()
            # All predecessors and successors
            if dependent.isalpha():
                nodes[current].prev.append(nodes[dependent])
                nodes[dependent].next.append(nodes[current])

    # Connect the start and end nodes
    for node in filter(lambda node: node.activity not in ["Start", "End"], nodes.values()):
        if not node.next:
            node.next.append(nodes["End"])
            nodes["End"].prev.append(node)

        if not node.prev:
            node.prev.append(nodes["Start"])
            nodes["Start"].next.append(node)

    return nodes


# Forward traversing - to set ES and EF values
def forward(end: Node):
    node = end
    max_es = 0

    for predecessor in node.prev:
        forward(end=predecessor)
        max_es = max(max_es, predecessor.ef)
    # Early start is the maximum value of predecessors EF values
    node.es = max_es
    node.ef = node.es + node.duration


# Backward traversing - to set LS and LF values
def backward(end: Node):
    node = end
    min_lf = sys.maxsize
    min_ff = sys.maxsize

    for successor in node.next:
        if successor.activity != "End":
            backward(end=successor)
        min_lf = min(min_lf, successor.ls)
        min_ff = min(min_lf, successor.es - node.ef)

    # Late finish is the minimum value of successors LS values
    node.lf = min_lf
    node.ls = node.lf - node.duration

    # Set is_critical to True if node is critical
    node.tf = node.ls - node.es

    if node.tf == 0:
        node.is_critical = "Yes"

    node.ff = min_ff


#
def get_df_from_nodes(nodes):
    df_ = []
    for node in nodes.values():
        node_str = repr(node)
        node_dict = json.loads(node_str)
        df_.append(node_dict)

    df = pd.DataFrame.from_dict(df_)
    df.drop(columns=["prev", "next"], axis=1, inplace=True)
    df = df[~df["Activity"].isin(["Start", "End"])]
    # df['Color'] = df.apply(color, axis=1)    

    return df


# Save Gantt-Chart to directory
def plot_lob_diagram(df, title, proj_start, proj_end, units, xlabel, ylabel, str_):
    fig, ax = plt.subplots(1, figsize=(25, 10))

    # Plot Legends
    df_dict = df.to_dict()
    activities = df_dict["Activity"].values()
    c_dict = {activity: "#" + "%06x" % random.randint(0, 0xFFFFFF) for activity in activities}

    legend_elements = [Patch(facecolor=c_dict[i], label=i) for i in c_dict]
    plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), fancybox=True, shadow=True)

    # Plot Ticks
    xticks = np.arange(0, proj_end + 10, 5)
    xticks_labels = np.arange(proj_start, proj_end + 10, 5)
    yticks = np.arange(0, units + 2, 1)
    yticks_labels = np.arange(0, units + 2, 1)

    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks_labels[::1], fontsize=15)

    ax.set_yticks(yticks)
    ax.set_yticklabels(yticks_labels[::1], fontsize=15)

    # Add Parallelograms
    for index, row in df.iterrows():
        x = []
        y = [0, 0, units, units]
        x.append(row["FUS"])
        x.append(row["FUF"])
        x.append(row["LUF"])
        x.append(row["LUS"])

        ax.add_patch(patches.Polygon(xy=list(zip(x, y)), fill=False, color=c_dict[row["Activity"]],
                                     linewidth=3))

    plt.title(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=20)
    plt.grid()

    # Save the figure
    if not os.path.exists(f'./{title[0:2]}_Plots/'):
        os.makedirs(f'./{title[0:2]}_Plots/')

    plt.suptitle(str_, fontsize=15)

    plt.savefig(f'./{title[0:2]}_Plots/' + title + ".png")

    # Clear the current figure
    plt.clf()


# Generate a PNG from pandas dataframe
def save_table(df, title):

    fig, ax = plt.subplots(1, figsize=(20, 7))
    # hide axes
    # fig.patch.set_visible(False)
    ax.axis('off')
    ax.patch.set_facecolor('white')

    # Create a table from df values
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    table.auto_set_font_size(True)

    # table.set_fontsize(9)
    table.auto_set_column_width(col=list(range(len(df.columns))))
    table.scale(1, 2)

    # plt.autoscale()
    plt.savefig(f'./{title[0:2]}_Plots/' + title + ".png", transparent=False)

    plt.clf()

import math
CSV_PATH = "/content/Assignment_5.csv"


def line_of_balance(productivity, units, nodes, activity):
    for node in nodes.values():
        if node.get_activity() == "Start" or node.get_activity() == "End":
            continue

        if not activity:
            # Theoretical Teams
            th_teams = float("{:.4f}".format(
                float(node.get_duration()) * (1 / productivity))
            )
            node.set_th_teams(th_teams)

            # Actual Teams
            if node.get_th_teams() < 1:
                node.set_ac_teams(1)
            else:
                node.set_ac_teams(math.floor(node.get_th_teams()))

            # Theoretical Productivity
            th_prod = float("{:.4f}".format(
                float(node.get_th_teams()) / float(node.get_duration()))
            )
            node.set_th_prod(th_prod)

            # Actual Productivity
            ac_prod = float("{:.4f}".format(
                float(node.get_ac_teams()) / float(node.get_duration()))
            )
            node.set_ac_prod(ac_prod)

        # List of previous activities
        prev = node.get_prev()
        num_prev = len(prev)

        # FUS, FUF, LUS, LUF Calculations
        prev_slope = 0.0
        prev_fuf = 0.0
        prev_luf = 0.0
        multiple = True
        first_node = False

        for prev_activity in prev:
            if num_prev == 1:
                # Node has 1 predecessor
                multiple = False

                # Check if this is the first node
                if prev_activity.get_activity() == "Start":
                    fus = float("{:.4f}".format(0.0))
                    fuf = float("{:.4f}".format(fus + node.get_duration()))
                    lus = float("{:.4f}".format(fus + ((units - 1) / node.get_ac_prod())))
                    luf = float("{:.4f}".format(lus + node.get_duration()))

                    node.set_fus(fus)
                    node.set_fuf(fuf)
                    node.set_lus(lus)
                    node.set_luf(luf)

                    first_node = True

                else:
                    # Set the previous_fuf and previous_luf values
                    prev_slope = float("{:.4f}".format(prev_activity.get_ac_prod()))
                    prev_fuf = float("{:.4f}".format(prev_activity.get_fuf()))
                    prev_luf = float("{:.4f}".format(prev_activity.get_luf()))

            else:
                # Current node has more than 1 predecessors
                # Get max(predecessor_fuf) and max(predecessor_luf)
                prev_fuf = float("{:.4f}".format(max(prev_fuf, prev_activity.get_fuf())))
                prev_luf = float("{:.4f}".format(max(prev_luf, prev_activity.get_luf())))

        if first_node:
            continue

        if multiple:
            # Define slope between prev_luf, prev_fuf
            prev_slope = float("{:.4f}".format((units - 1) / (prev_luf - prev_fuf)))

        # Compare the slopes to define bottle-neck location
        if node.get_ac_prod() <= prev_slope:
            # Bottle-neck is downwards
            fus = float("{:.4f}".format(prev_fuf + node.get_buffer()))
            fuf = float("{:.4f}".format(fus + node.get_duration()))
            lus = float("{:.4f}".format(fus + (units - 1) / node.get_ac_prod()))
            luf = float("{:.4f}".format(lus + node.get_duration()))

        else:
            # Bottle-neck is upwards
            lus = float("{:.4f}".format(prev_luf + node.get_buffer()))
            luf = float("{:.4f}".format(lus + node.get_duration()))
            fus = float("{:.4f}".format(lus - ((units - 1) / node.get_ac_prod())))
            fuf = float("{:.4f}".format(fus + node.get_duration()))

        node.set_fus(fus)
        node.set_fuf(fuf)
        node.set_lus(lus)
        node.set_luf(luf)

    return nodes


if __name__ == "__main__":
    # User Input - duration of project
    desired_duration = int(input("Please enter the desired duration (deadline) of the project: "))
    # User Input - units
    units = float(input("Please enter the desired number of units: "))

    csv = read_csv(CSV_PATH)
    nodes = generate_nodes(csv, True)

    # Forward and Backward Pass
    start = nodes["Start"]
    end = nodes["End"]

    # Populate earliest start and finish by recurring until the end node
    forward(end=end)

    # Update "end" late start and finish values
    end.ls = end.lf = end.es

    # Populate late start and finish by recurring until the start node
    backward(end=start)

    # Get initial productivity for trial/error
    productivity = float("{:.4f}".format((desired_duration - end.es) / (units - 1)))

    nodes = generate_nodes(csv, False)

    # First LOB Analysis - get the project duration
    nodes = line_of_balance(productivity, units, nodes, False)
    df = get_df_from_nodes(nodes)
    df.reset_index(drop=True, inplace=True)
    df["LUF"] = pd.to_numeric(df["LUF"])
    proj_duration = df["LUF"].max()

    print("Desired Project Duration: ", desired_duration)
    print("Initial Project Duration: ", proj_duration)
    print("Initial Productivity: ", productivity)
    print("============================================================")

    low = 0
    med = productivity
    high = 2 * med - low

    num_iter = 0
    while abs(proj_duration - desired_duration) > 0.1:

        if proj_duration < desired_duration:
            # productivity must be decreased so that project_duration increases => (1/productivity) increases
            # Set low = current productivity
            low = med
            # Medium is the new productivity
            med = float((low + high) / 2)

            if low == med:
                # Break as soon as productivity cannot be changed
                break

            print("Current Productivity: ", low)
            print("Current Productivity is too low. Set low = current_productivity.")
            print("High remains the same")
            print("Low: ", low, "\nMedium: ", med, "\nHigh: ", high)
            print("New Productivity: ", med)

        else:
            # productivity must be increased so that project_duration decreases => (1/productivity) decreases
            # Set high = current productivity
            high = med
            # Medium is the new productivity
            med = float((low + high) / 2)

            if high == med:
                # Break as soon as productivity cannot be changed
                break

            print("Current Productivity: ", high)
            print("Current Productivity is too high. Set high = current_productivity.")
            print("Low remains the same")
            print("Low: ", low, "\nMedium: ", med, "\nHigh: ", high)
            print("New Productivity: ", med)

        nodes = line_of_balance(med, units, nodes, False)
        df = get_df_from_nodes(nodes)
        df.reset_index(drop=True, inplace=True)
        df["LUF"] = pd.to_numeric(df["LUF"])
        proj_duration = df["LUF"].max()
        print("Project Duration: ", proj_duration, "For Productivity: ", med)
        print("============================================================")

        num_iter += 1

    # Plot the LOB Diagram/Table
    df = get_df_from_nodes(nodes)
    df.drop(columns=["ES", "EF", "LS", "LF", "TF", "FF"], axis=1, inplace=True)
    print(df.head(15))
    df["LUF"] = pd.to_numeric(df["LUF"])
    start_date = 0
    end_date = df["LUF"].max()
    str_ = f"Project Duration: {end_date} days\n" \
           f"Desired Duration: {desired_duration}\n" \
           f"Distance From Given Deadline: {abs(desired_duration - end_date)} day(s)"
    plot_lob_diagram(df, "Q1 LOB Diagram", start_date, end_date, units, "Duration (Days)", "Units", str_)

    save_table(df, "Q1 LOB Table")

    # For each iteration, update an activity's Ac_teams by 1
    activity_ = None
    refined_duration = proj_duration

    for node in nodes.values():
        if node.get_activity() in ["Start", "End"]:
            continue

        # Increase the Ac_teams by 1
        node.set_ac_teams(node.get_ac_teams() + 1)
        # Actual Productivity
        ac_prod = float("{:.4f}".format(
            float(node.get_ac_teams()) / float(node.get_duration()))
        )
        node.set_ac_prod(ac_prod)

        # Perform LOB Analysis - get the project duration
        nodes = line_of_balance(productivity, units, nodes, True)

        df = get_df_from_nodes(nodes)
        df.reset_index(drop=True, inplace=True)
        df["LUF"] = pd.to_numeric(df["LUF"])
        proj_duration = df["LUF"].max()

        if proj_duration > refined_duration and abs(proj_duration - refined_duration) < 0.1:
            refined_duration = proj_duration
            activity_ = node.get_activity()

        # Reset the node's Ac_teams
        node.set_ac_teams(node.get_ac_teams() - 1)
        # Reset the nodee's Ac_prod
        ac_prod = float("{:.4f}".format(
            float(node.get_ac_teams()) / float(node.get_duration()))
        )
        node.set_ac_prod(ac_prod)

    if activity_ is None:
        print("No Improvements")

    else:
        print(f"Refined Duration: {refined_duration} by updating Activity {activity_}'s Actual_Team by 1.0")
        print(f"Distance From Given Deadline: {abs(desired_duration - refined_duration)} day(s)")


Please enter the desired duration (deadline) of the project: 100
Please enter the desired number of units: 10
Desired Project Duration:  100
Initial Project Duration:  152.9865
Initial Productivity:  4.1111
Current Productivity:  4.1111
Current Productivity is too high. Set high = current_productivity.
Low remains the same
Low:  0 
Medium:  2.05555 
High:  4.1111
New Productivity:  2.05555
Project Duration:  91.8061 For Productivity:  2.05555
Current Productivity:  2.05555
Current Productivity is too low. Set low = current_productivity.
High remains the same
Low:  2.05555 
Medium:  3.0833250000000003 
High:  4.1111
New Productivity:  3.0833250000000003
Project Duration:  123.0 For Productivity:  3.0833250000000003
Current Productivity:  3.0833250000000003
Current Productivity is too high. Set high = current_productivity.
Low remains the same
Low:  2.05555 
Medium:  2.5694375000000003 
High:  3.0833250000000003
New Productivity:  2.5694375000000003
Project Duration:  129.0 For Productiv

<Figure size 1800x720 with 0 Axes>

<Figure size 1440x504 with 0 Axes>